In [10]:
import pandas as pd
import geopandas
import rasterio as rio
data = rio.open('../preprocessed_data/dimensionality_reduction/geotiff/erosion_take_2.tif')


print(data.transform)
lat_long_NW = data.transform * (0, 0)
print(lat_long_NW)
lat_long_SE = data.transform * (data.width, data.height)
print(lat_long_SE)
print(data.crs)

# read bands
array = data.read()

data_df = pd.DataFrame()

array_num = 0
band_num = 1
min_lat = lat_long_NW[0]
min_long = lat_long_SE[1]
max_lat = lat_long_SE[0]
max_long = lat_long_NW[1]


for num in range(358):
    data_df['band' +str(band_num)]=array[array_num].ravel()
    array_num += 1
    band_num += 1

data_df.head()

| 5.10, 0.00, 448208.40|
| 0.00,-5.10, 6481850.87|
| 0.00, 0.00, 1.00|
(448208.4004, 6481850.874)
(449208.0004, 6480841.074)
EPSG:32630


/tmp/ipykernel_1981/2575087084.py:38: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  data_df['band' +str(band_num)]=array[array_num].ravel()


,band1,band2,band3,band4,band5,band6,band7,band8,band9,band10,...,band349,band350,band351,band352,band353,band354,band355,band356,band357,band358
0,-0.000152,0.000986,0.004008,0.005472,0.005756,0.007789,0.009954,0.011664,0.013060,0.014920,...,0.064053,0.063936,0.061351,0.065499,0.062266,0.063747,0.060986,0.059916,0.061251,0.058821
1,0.000510,0.000955,0.004493,0.004698,0.006199,0.008139,0.010138,0.011971,0.014497,0.015473,...,0.064242,0.064434,0.062267,0.062596,0.063473,0.060694,0.062735,0.062540,0.061148,0.063097
2,0.000565,0.001765,0.003405,0.005407,0.005677,0.008092,0.010280,0.011298,0.014343,0.015980,...,0.065214,0.065666,0.064580,0.068078,0.066012,0.062762,0.061232,0.066458,0.060376,0.063481
3,0.000197,0.001537,0.003782,0.005053,0.006393,0.007700,0.010634,0.011635,0.013751,0.015548,...,0.065201,0.066174,0.065376,0.065658,0.066481,0.064339,0.062169,0.064833,0.062855,0.061849
4,0.000440,0.001375,0.003630,0.005567,0.006266,0.008108,0.010253,0.010777,0.013712,0.014617,...,0.063889,0.063503,0.062631,0.063027,0.059421,0.060017,0.062054,0.061573,0.059307,0.060271


In [12]:
data_df["POINTID"] = ""


index_num = 0
point_num = 1

for index, row in data_df.iterrows():
    data_df.iloc[index_num, 358] = point_num
    point_num += 1
    index_num += 1
    
data_df.head()

/tmp/ipykernel_1981/691728217.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  data_df["POINTID"] = ""


,band1,band2,band3,band4,band5,band6,band7,band8,band9,band10,...,band350,band351,band352,band353,band354,band355,band356,band357,band358,POINTID
0,-0.000152,0.000986,0.004008,0.005472,0.005756,0.007789,0.009954,0.011664,0.013060,0.014920,...,0.063936,0.061351,0.065499,0.062266,0.063747,0.060986,0.059916,0.061251,0.058821,1
1,0.000510,0.000955,0.004493,0.004698,0.006199,0.008139,0.010138,0.011971,0.014497,0.015473,...,0.064434,0.062267,0.062596,0.063473,0.060694,0.062735,0.062540,0.061148,0.063097,2
2,0.000565,0.001765,0.003405,0.005407,0.005677,0.008092,0.010280,0.011298,0.014343,0.015980,...,0.065666,0.064580,0.068078,0.066012,0.062762,0.061232,0.066458,0.060376,0.063481,3
3,0.000197,0.001537,0.003782,0.005053,0.006393,0.007700,0.010634,0.011635,0.013751,0.015548,...,0.066174,0.065376,0.065658,0.066481,0.064339,0.062169,0.064833,0.062855,0.061849,4
4,0.000440,0.001375,0.003630,0.005567,0.006266,0.008108,0.010253,0.010777,0.013712,0.014617,...,0.063503,0.062631,0.063027,0.059421,0.060017,0.062054,0.061573,0.059307,0.060271,5


In [13]:
geometry_file = pd.read_csv('../preprocessed_data/dimensionality_reduction/shape/erosion_geometry_take_2.csv')

geometry_file

,wkt_geom,VALUE,POINTID
0,Point (448210.95043433899991214 6481848.324042...,-0.000152,1
1,Point (448216.05043433897662908 6481848.324042...,0.000510,2
2,Point (448221.15043433895334601 6481848.324042...,0.000565,3
3,Point (448226.25043433893006295 6481848.324042...,0.000197,4
4,Point (448231.35043433890677989 6481848.324042...,0.000440,5
...,...,...,...
38803,Point (449185.05043433455284685 6480843.624042...,0.001394,38804
38804,Point (449190.15043433452956378 6480843.624042...,0.001595,38805
38805,Point (449195.25043433450628072 6480843.624042...,0.003444,38806
38806,Point (449200.35043433448299766 6480843.624042...,0.003120,38807


In [14]:
data_gdf = pd.merge(data_df, geometry_file, on="POINTID")

data_gdf = data_gdf.rename(columns={"wkt_geom": "geometry"})

data_gdf.head()


,band1,band2,band3,band4,band5,band6,band7,band8,band9,band10,...,band352,band353,band354,band355,band356,band357,band358,POINTID,geometry,VALUE
0,-0.000152,0.000986,0.004008,0.005472,0.005756,0.007789,0.009954,0.011664,0.013060,0.014920,...,0.065499,0.062266,0.063747,0.060986,0.059916,0.061251,0.058821,1,Point (448210.95043433899991214 6481848.324042...,-0.000152
1,0.000510,0.000955,0.004493,0.004698,0.006199,0.008139,0.010138,0.011971,0.014497,0.015473,...,0.062596,0.063473,0.060694,0.062735,0.062540,0.061148,0.063097,2,Point (448216.05043433897662908 6481848.324042...,0.000510
2,0.000565,0.001765,0.003405,0.005407,0.005677,0.008092,0.010280,0.011298,0.014343,0.015980,...,0.068078,0.066012,0.062762,0.061232,0.066458,0.060376,0.063481,3,Point (448221.15043433895334601 6481848.324042...,0.000565
3,0.000197,0.001537,0.003782,0.005053,0.006393,0.007700,0.010634,0.011635,0.013751,0.015548,...,0.065658,0.066481,0.064339,0.062169,0.064833,0.062855,0.061849,4,Point (448226.25043433893006295 6481848.324042...,0.000197
4,0.000440,0.001375,0.003630,0.005567,0.006266,0.008108,0.010253,0.010777,0.013712,0.014617,...,0.063027,0.059421,0.060017,0.062054,0.061573,0.059307,0.060271,5,Point (448231.35043433890677989 6481848.324042...,0.000440


In [15]:
from shapely.geometry import Point

data_gdf['geometry'] = geopandas.GeoSeries.from_wkt(data_gdf['geometry'])

gdf = geopandas.GeoDataFrame(data_gdf, geometry='geometry')



In [16]:
gdf.describe()

,band1,band2,band3,band4,band5,band6,band7,band8,band9,band10,...,band350,band351,band352,band353,band354,band355,band356,band357,band358,VALUE
count,38808.000000,38808.000000,38808.000000,38808.000000,38808.000000,38808.000000,38808.000000,38808.000000,38808.000000,38808.000000,...,38808.000000,38808.000000,38808.000000,38808.000000,38808.000000,38808.000000,38808.000000,38808.000000,38808.000000,38808.000000
mean,0.001297,0.001677,0.003559,0.004432,0.005364,0.007740,0.009786,0.011503,0.013664,0.015692,...,0.080611,0.079219,0.080591,0.078530,0.078455,0.078859,0.079439,0.078180,0.078065,0.001297
std,0.000936,0.000827,0.000700,0.000726,0.000639,0.000645,0.000741,0.000893,0.001038,0.001241,...,0.013954,0.013813,0.014004,0.013844,0.013947,0.013963,0.014318,0.014277,0.014482,0.000936
min,-0.004076,-0.003392,-0.001155,0.001377,0.002809,0.004870,0.006602,0.007268,0.008595,0.009625,...,0.008793,0.009643,0.010310,0.007019,0.010003,0.010855,0.010596,0.007904,0.008847,-0.004076
25%,0.000801,0.001219,0.003146,0.003990,0.004977,0.007347,0.009317,0.010937,0.013047,0.014975,...,0.072755,0.071327,0.072478,0.070481,0.070205,0.070555,0.070967,0.069630,0.069359,0.000801
50%,0.001186,0.001568,0.003460,0.004282,0.005245,0.007649,0.009742,0.011509,0.013694,0.015730,...,0.082862,0.081454,0.082905,0.080784,0.080698,0.081097,0.081747,0.080462,0.080362,0.001186
75%,0.001749,0.002102,0.003910,0.004755,0.005635,0.008054,0.010181,0.012015,0.014293,0.016462,...,0.090504,0.089093,0.090637,0.088499,0.088503,0.089001,0.089835,0.088642,0.088695,0.001749
max,0.012067,0.011616,0.012034,0.014202,0.015632,0.020067,0.023420,0.027731,0.029528,0.031979,...,0.181333,0.172922,0.177517,0.169230,0.172499,0.174656,0.175507,0.172793,0.175431,0.012067


In [17]:
gdf.to_file("erosion_gdf_take_2.shp")


In [18]:
gdf.to_csv("erosion_gdf_take_2.csv")